---

In [1]:
# Основные импорты
"""
Сравнение OCR/VLM моделей для распознавания текста из изображений.

Требования:
- Запущенные vLLM серверы на соответствующих портах
- Изображения для тестирования в указанной папке

Структура:
1. Импорты и настройки
2. Конфигурация (пути, параметры)
3. Вспомогательные функции
4. Тестирование каждой модели отдельно
5. Сбор и анализ результатов
"""

import sys
from pathlib import Path

# Основные импорты
import base64
import logging
import time
import json
from io import BytesIO
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, asdict
from datetime import datetime


# Работа с изображениями
from PIL import Image
from pdf2image import convert_from_path

# API клиент
from openai import OpenAI

# Визуализация и анализ
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# ПАРАЛЛЕЛЬНАЯ ОБРАБОТКА
# ============================================================================
from concurrent.futures import ThreadPoolExecutor, as_completed
# ThreadPoolExecutor - пул потоков для параллельных API-запросов
# as_completed - итератор для получения результатов по мере готовности

import threading
# threading.Lock - для thread-safe операций (при необходимости)

# ============================================================================
# ВНЕШНИЕ БИБЛИОТЕКИ
# ============================================================================
from PIL import Image
import requests
from tqdm import tqdm
from openai import OpenAI

# ============================================================================
# НАСТРОЙКА ЛОГИРОВАНИЯ
# ============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('ocr_processing.log', encoding='utf-8')
    ]
)
logger = logging.getLogger(__name__)

# ============================================================================
# ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ
# ============================================================================
# Добавить корень проекта в PYTHONPATH
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name != "Team_5_AI_Tutor":
    PROJECT_ROOT = PROJECT_ROOT.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data"
IMAGES_DIR = PROJECT_ROOT / "notebooks" / "vlm-ingestion-pipeline" / "test_images"
OUTPUT_DIR = DATA_DIR / "test_results"

# Создаём директории если не существуют
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"   ThreadPoolExecutor: готов для I/O-bound задач")
print(f"   threading: доступен для синхронизации")
print("✅ Все зависимости загружены")
print(f"📁 Корень проекта: {PROJECT_ROOT}")


   ThreadPoolExecutor: готов для I/O-bound задач
   threading: доступен для синхронизации
✅ Все зависимости загружены
📁 Корень проекта: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor


In [2]:
#Конфигурация путей, параметров и моделей.


# ==================== ПУТИ ====================
# Корневая папка с тестовыми данными
TEST_ROOT = PROJECT_ROOT / "notebooks" / "vlm-ingestion-pipeline" / "test_images"

# Режимы загрузки изображений:
# "flat" - все изображения в одной папке (data/test/*.png)
# "nested" - изображения в подпапках (data/test/book1/*.png, data/test/book2/*.png)
# "recursive" - рекурсивный поиск по всем вложенным папкам
LOAD_MODE = "nested"  # Измените на "flat" или "recursive" если нужно

# Папка для сохранения результатов
OUTPUT_DIR = PROJECT_ROOT / "data" / "test_results"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Имя эксперимента для сохранения результатов
EXPERIMENT_NAME = f"ocr_comparison_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# ==================== ПАРАМЕТРЫ ОБРАБОТКИ ====================
# Максимальное количество изображений для теста
MAX_IMAGES_PER_FOLDER = None  # None = все изображения
MAX_TOTAL_IMAGES = None  # None = без ограничений на общее количество

# Расширения файлов изображений
IMAGE_EXTENSIONS = ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG', '.tiff', '.bmp']

# Таймаут для API запросов (секунды)
API_TIMEOUT = 180

# Сохранять метаданные о папках/книгах
PRESERVE_FOLDER_STRUCTURE = True  # Включить группировку по папкам

# ==================== МОДЕЛИ ====================
MODELS_CONFIG = {
    "HunyuanOCR": {
        "port": 8000,
        "model_path": "/models/HunyuanOCR",
        "description": "Tencent OCR specialist",
        "prompts": {
            "default": "OCR this text. Extract all text accurately.",
            "detailed": "Please extract all text from this image, preserving the original formatting and structure.",
            "russian": "Распознай весь текст на этом изображении с максимальной точностью.",
            "short": "OCR"
        },
        "enabled": True
    },
    
    "Qwen3-VL-8B": {
        "port": 8001,
        "model_path": "/models/Qwen3-VL-8B-Instruct",
        "description": "Alibaba general-purpose VLM",
        "prompts": {
            "default": "Extract all text from this image.",
            "detailed": "Please read and transcribe all visible text in this image.",
            "russian": "Извлеки весь текст из этого изображения.",
            "short": "Read text"
        },
        "enabled": True
    },
    
    "LightOnOCR-1B": {
        "port": 8002,
        "model_path": "/models/LightOnOCR-1B-1025",
        "description": "LightOn OCR specialist (fastest)",
        "prompts": {
            "default": "OCR",
            "detailed": "OCR this document",
            "russian": "OCR",
            "short": "OCR"
        },
        "enabled": True
    },
    
    "Nemotron-Nano-VL-8B": {
        "port": 8003,
        "model_path": "/models/Llama-3.1-Nemotron-Nano-VL-8B-V1",
        "description": "NVIDIA document intelligence",
        "prompts": {
            "default": "Extract all text from this document image.",
            "detailed": "Please carefully read and extract all text content from this document, maintaining the original structure.",
            "russian": "Извлеки весь текст из этого документа.",
            "short": "Extract text"
        },
        "enabled": True
    },
    
    "Jina-VLM": {
        "port": 8004,
        "model_path": "/models/jina-vlm",
        "description": "Jina AI multilingual VLM",
        "prompts": {
            "default": "Please extract all text from this image.",
            "detailed": "Read and transcribe all visible text in this image, preserving formatting.",
            "russian": "Пожалуйста, извлеки весь текст из этого изображения.",
            "short": "Extract text"
        },
        "enabled": True
    }
}

# Выбор типа промпта: "default", "detailed", "russian", "short"
PROMPT_TYPE = "default"

print("✅ Конфигурация загружена")
print(f"📂 Корневая папка: {TEST_ROOT}")
print(f"📁 Режим загрузки: {LOAD_MODE}")
print(f"💾 Результаты: {OUTPUT_DIR}")
print(f"🎯 Эксперимент: {EXPERIMENT_NAME}")
print(f"\n📊 Активные модели:")
for name, config in MODELS_CONFIG.items():
    status = "✅" if config["enabled"] else "⏸️"
    print(f"  {status} {name}: {config['description']} (порт {config['port']})")


✅ Конфигурация загружена
📂 Корневая папка: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/notebooks/vlm-ingestion-pipeline/test_images
📁 Режим загрузки: nested
💾 Результаты: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/data/test_results
🎯 Эксперимент: ocr_comparison_20251220_012303

📊 Активные модели:
  ✅ HunyuanOCR: Tencent OCR specialist (порт 8000)
  ✅ Qwen3-VL-8B: Alibaba general-purpose VLM (порт 8001)
  ✅ LightOnOCR-1B: LightOn OCR specialist (fastest) (порт 8002)
  ✅ Nemotron-Nano-VL-8B: NVIDIA document intelligence (порт 8003)
  ✅ Jina-VLM: Jina AI multilingual VLM (порт 8004)


In [3]:
# Структуры данных для хранения результатов с метаданными о папках.

@dataclass
class OCRResult:
    """
    Результат OCR для одного изображения.
    
    Thread-safety: ✅ Immutable после создания
    """
    model_name: str           # ← Переместили наверх для логичности
    folder_name: str          # Название папки/книги
    filename: str             # Имя файла (page_001.png)
    page_number: int          # ← ИЗМЕНЕНО: было page_num
    text: str                 # Распознанный текст
    tokens: int               # ← ИЗМЕНЕНО: было tokens_used
    processing_time: float    # Время обработки (секунды)
    text_length: int          # Длина текста (автозаполняется)
    timestamp: str            # ISO timestamp (автозаполняется)
    error: Optional[str] = None  # Текст ошибки
    
    # Дополнительные поля (опционально, для совместимости)
    image_name: str = ""      # Дублирует filename
    image_path: str = ""      # Полный путь
    prompt: str = ""          # Использованный промпт
    
    def __post_init__(self):
        """Автозаполнение полей после создания объекта"""
        # Автозаполнение timestamp
        if not self.timestamp:
            self.timestamp = datetime.now().isoformat()
        
        # Автозаполнение text_length
        if self.text_length == 0:
            self.text_length = len(self.text)
        
        # Дублирование filename в image_name для совместимости
        if not self.image_name:
            self.image_name = self.filename


@dataclass
class ModelStats:
    """
    Статистика работы модели.
    
    Thread-safety: ✅ Создаётся после завершения всех потоков
    """
    model_name: str
    total_images: int
    folders_processed: int     # Количество папок
    total_time: float          # Общее время (секунды)
    avg_time_per_image: float  # Среднее время на изображение
    total_tokens: int          # Всего токенов
    total_chars: int           # Всего символов
    success_rate: float        # Процент успешных (0-100)
    errors: int                # Количество ошибок
    throughput: float          # Изображений в секунду
    
    # Дополнительные поля
    avg_tokens_per_image: float = 0.0
    avg_chars_per_image: float = 0.0
    prompt_used: str = ""


@dataclass
class FolderStats:
    """
    Статистика по одной папке/книге.
    
    Thread-safety: ✅ Создаётся после завершения всех потоков
    """
    folder_name: str
    total_images: int
    total_time: float
    avg_time_per_image: float
    total_chars: int
    avg_chars_per_image: float
    success_rate: float = 100.0


# Глобальное хранилище результатов
ALL_RESULTS = {}      # {model_name: [OCRResult, ...]}
FOLDER_GROUPS = {}    # {folder_name: [image_paths, ...]}
IMAGE_PATHS = []      # Список всех путей к изображениям

print("✅ Классы данных определены (обновлённая версия)")
print("   • OCRResult: унифицированные имена полей")
print("   • ModelStats: расширенная статистика")
print("   • FolderStats: статистика по папкам")


✅ Классы данных определены (обновлённая версия)
   • OCRResult: унифицированные имена полей
   • ModelStats: расширенная статистика
   • FolderStats: статистика по папкам


In [4]:
# Вспомогательные функции с поддержкой вложенных папок.


def load_images_flat(
    root_dir: Path,
    extensions: List[str] = IMAGE_EXTENSIONS,
    max_images: Optional[int] = None
) -> Tuple[List[Path], Dict[str, List[Path]]]:
    """
    Загрузка изображений из одной папки (плоская структура).
    
    Returns:
        (список путей, {folder_name: [paths]})
    """
    image_paths = []
    
    for ext in extensions:
        image_paths.extend(root_dir.glob(f"*{ext}"))
    
    image_paths = sorted(image_paths)
    
    if max_images:
        image_paths = image_paths[:max_images]
    
    folder_groups = {"root": image_paths}
    
    logger.info(f"✅ Найдено {len(image_paths)} изображений в {root_dir}")
    
    return image_paths, folder_groups


def load_images_nested(
    root_dir: Path,
    extensions: List[str] = IMAGE_EXTENSIONS,
    max_per_folder: Optional[int] = None,
    max_total: Optional[int] = None
) -> Tuple[List[Path], Dict[str, List[Path]]]:
    """
    Загрузка изображений из подпапок (вложенная структура).
    
    Структура:
    root_dir/
    ├── folder1/
    │   ├── image1.png
    │   └── image2.png
    └── folder2/
        └── image3.png
    
    Returns:
        (список всех путей, {folder_name: [paths]})
    """
    folder_groups = {}
    all_paths = []
    
    # Найти все подпапки
    subfolders = [d for d in root_dir.iterdir() if d.is_dir()]
    
    if not subfolders:
        logger.warning(f"⚠️  Подпапки не найдены в {root_dir}")
        return load_images_flat(root_dir, extensions, max_total)
    
    logger.info(f"📁 Найдено {len(subfolders)} подпапок")
    
    for subfolder in sorted(subfolders):
        folder_name = subfolder.name
        folder_images = []
        
        # Загрузка изображений из подпапки
        for ext in extensions:
            folder_images.extend(subfolder.glob(f"*{ext}"))
        
        folder_images = sorted(folder_images)
        
        # Ограничение на папку
        if max_per_folder:
            folder_images = folder_images[:max_per_folder]
        
        if folder_images:
            folder_groups[folder_name] = folder_images
            all_paths.extend(folder_images)
            logger.info(f"  📂 {folder_name}: {len(folder_images)} изображений")
    
    # Ограничение общее
    if max_total and len(all_paths) > max_total:
        all_paths = all_paths[:max_total]
        logger.info(f"⚠️  Ограничение до {max_total} изображений")
    
    logger.info(f"✅ Всего загружено {len(all_paths)} изображений из {len(folder_groups)} папок")
    
    return all_paths, folder_groups


def load_images_recursive(
    root_dir: Path,
    extensions: List[str] = IMAGE_EXTENSIONS,
    max_total: Optional[int] = None
) -> Tuple[List[Path], Dict[str, List[Path]]]:
    """
    Рекурсивная загрузка изображений из всех вложенных папок.
    
    Returns:
        (список всех путей, {relative_folder_path: [paths]})
    """
    folder_groups = {}
    all_paths = []
    
    for ext in extensions:
        for img_path in root_dir.rglob(f"*{ext}"):
            if img_path.is_file():
                # Относительный путь папки
                folder_rel = img_path.parent.relative_to(root_dir)
                folder_name = str(folder_rel) if str(folder_rel) != "." else "root"
                
                if folder_name not in folder_groups:
                    folder_groups[folder_name] = []
                
                folder_groups[folder_name].append(img_path)
                all_paths.append(img_path)
    
    # Сортировка
    all_paths = sorted(all_paths)
    for folder in folder_groups:
        folder_groups[folder] = sorted(folder_groups[folder])
    
    # Ограничение
    if max_total and len(all_paths) > max_total:
        all_paths = all_paths[:max_total]
    
    logger.info(f"✅ Рекурсивно найдено {len(all_paths)} изображений в {len(folder_groups)} папках")
    
    return all_paths, folder_groups


def image_to_base64(image: Image.Image) -> str:
    """Конвертация PIL Image в base64."""
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()


def check_model_availability(port: int, timeout: int = 5) -> bool:
    """Проверка доступности модели."""
    try:
        client = OpenAI(
            base_url=f"http://localhost:{port}/v1",
            api_key="EMPTY",
            timeout=timeout
        )
        client.models.list()
        return True
    except:
        return False


def ocr_single_image(
    image_path: Path,
    folder_name: str,
    model_name: str,
    model_config: Dict,
    prompt_type: str = "default",
    page_num: Optional[int] = None
) -> OCRResult:
    """
    OCR одного изображения с сохранением метаданных о папке.
    
    Args:
        image_path: Путь к изображению
        folder_name: Название папки/книги
        model_name: Название модели
        model_config: Конфигурация модели
        prompt_type: Тип промпта
        page_num: Номер страницы
    """
    try:
        image = Image.open(image_path)
    except Exception as e:
        logger.error(f"Ошибка загрузки {image_path}: {e}")
        return OCRResult(
            image_name=image_path.name,
            image_path=str(image_path),
            folder_name=folder_name,
            page_num=page_num or 0,
            text="",
            processing_time=0.0,
            model_name=model_name,
            prompt="",
            error=f"Ошибка загрузки: {str(e)}"
        )
    
    port = model_config["port"]
    model_path = model_config["model_path"]
    prompt = model_config["prompts"].get(prompt_type, model_config["prompts"]["default"])
    
    client = OpenAI(
        base_url=f"http://localhost:{port}/v1",
        api_key="EMPTY",
        timeout=API_TIMEOUT
    )
    
    image_base64 = image_to_base64(image)
    start_time = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_path,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{image_base64}"
                            }
                        }
                    ]
                }
            ],
            # max_tokens=2048,
            temperature=0.1
        )
        
        processing_time = time.time() - start_time
        text = response.choices[0].message.content.strip()
        tokens = response.usage.total_tokens if response.usage else 0
        
        return OCRResult(
            image_name=image_path.name,
            image_path=str(image_path),
            folder_name=folder_name,
            page_num=page_num or 0,
            text=text,
            processing_time=processing_time,
            model_name=model_name,
            prompt=prompt,
            tokens_used=tokens,
            text_length=len(text)
        )
    
    except Exception as e:
        processing_time = time.time() - start_time
        logger.error(f"Ошибка OCR {model_name} на {image_path.name}: {e}")
        
        return OCRResult(
            image_name=image_path.name,
            image_path=str(image_path),
            folder_name=folder_name,
            page_num=page_num or 0,
            text="",
            processing_time=processing_time,
            model_name=model_name,
            prompt=prompt,
            tokens_used=0,
            text_length=0,
            error=str(e)
        )



def calculate_model_stats(results: List[OCRResult]) -> ModelStats:
    """Расчёт статистики для модели."""
    if not results:
        return None
    
    total_images = len(results)
    successful = [r for r in results if not r.error]
    errors = len(results) - len(successful)
    
    total_time = sum(r.processing_time for r in results)
    total_tokens = sum(r.tokens_used for r in results)
    total_chars = sum(r.text_length for r in results)
    
    # Количество уникальных папок
    folders = set(r.folder_name for r in results)
    
    return ModelStats(
        model_name=results[0].model_name,
        total_images=total_images,
        total_time=total_time,
        avg_time_per_image=total_time / total_images if total_images > 0 else 0,
        throughput=total_images / total_time if total_time > 0 else 0,
        total_tokens=total_tokens,
        avg_tokens_per_image=total_tokens / total_images if total_images > 0 else 0,
        total_chars=total_chars,
        avg_chars_per_image=total_chars / total_images if total_images > 0 else 0,
        success_rate=len(successful) / total_images * 100 if total_images > 0 else 0,
        errors=errors,
        prompt_used=results[0].prompt if results else "",
        folders_processed=len(folders)
    )


def calculate_folder_stats(results: List[OCRResult], folder_name: str) -> FolderStats:
    """Статистика по конкретной папке."""
    folder_results = [r for r in results if r.folder_name == folder_name]
    
    if not folder_results:
        return None
    
    total_time = sum(r.processing_time for r in folder_results)
    total_chars = sum(r.text_length for r in folder_results)
    
    return FolderStats(
        folder_name=folder_name,
        total_images=len(folder_results),
        total_time=total_time,
        avg_time_per_image=total_time / len(folder_results),
        total_chars=total_chars,
        avg_chars_per_image=total_chars / len(folder_results)
    )


print("✅ Вспомогательные функции определены")


✅ Вспомогательные функции определены


In [5]:
# Функции для сохранения результатов.


def save_results_to_files(
    results: List[OCRResult],
    output_dir: Path,
    model_name: str
) -> Dict[str, Path]:
    """
    Сохранение результатов OCR в файлы.
    
    Создаёт:
    1. Текстовые файлы по папкам
    2. JSON с полными результатами
    3. CSV со статистикой
    
    Returns:
        Словарь с путями к созданным файлам
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    saved_files = {}
    
    # Группировка по папкам
    results_by_folder = {}
    for result in results:
        if result.folder_name not in results_by_folder:
            results_by_folder[result.folder_name] = []
        results_by_folder[result.folder_name].append(result)
    
    # 1. Текстовые файлы по папкам
    for folder_name, folder_results in results_by_folder.items():
        # Безопасное имя файла
        safe_folder_name = folder_name.replace('/', '_').replace('\\', '_')
        txt_file = output_dir / f"{safe_folder_name}_{model_name}.txt"
        
        with open(txt_file, "w", encoding="utf-8") as f:
            f.write(f"{'='*80}\n")
            f.write(f"OCR РЕЗУЛЬТАТЫ: {folder_name}\n")
            f.write(f"Модель: {model_name}\n")
            f.write(f"Дата: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"{'='*80}\n\n")
            
            for result in sorted(folder_results, key=lambda r: r.page_num):
                f.write(f"\n{'─'*80}\n")
                f.write(f"СТРАНИЦА {result.page_num}\n")
                f.write(f"Изображение: {result.image_name}\n")
                f.write(f"Время обработки: {result.processing_time:.2f}с\n")
                f.write(f"Токены: {result.tokens_used}\n")
                f.write(f"Символов: {result.text_length}\n")
                
                if result.error:
                    f.write(f"ОШИБКА: {result.error}\n")
                else:
                    f.write(f"{'─'*80}\n\n")
                    f.write(result.text)
                    f.write("\n")
        
        saved_files[f"txt_{folder_name}"] = txt_file
        logger.info(f"💾 Текст сохранён: {txt_file.name}")
    
    # 2. JSON с полными данными
    json_file = output_dir / f"{model_name}_full_results.json"
    
    json_data = {
        "model": model_name,
        "timestamp": datetime.now().isoformat(),
        "total_images": len(results),
        "results": [
            {
                "image_name": r.image_name,
                "folder": r.folder_name,
                "page_num": r.page_num,
                "text": r.text,
                "processing_time": r.processing_time,
                "tokens_used": r.tokens_used,
                "text_length": r.text_length,
                "error": r.error
            }
            for r in results
        ]
    }
    
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    
    saved_files["json"] = json_file
    logger.info(f"💾 JSON сохранён: {json_file.name}")
    
    # 3. CSV статистика
    csv_file = output_dir / f"{model_name}_stats.csv"
    
    rows = []
    for result in results:
        rows.append({
            "folder": result.folder_name,
            "image": result.image_name,
            "page": result.page_number,
            "time_sec": result.processing_time,
            "tokens": result.tokens,
            "chars": result.text_length,
            "error": result.error or ""
        })
    
    df = pd.DataFrame(rows)
    df.to_csv(csv_file, index=False, encoding="utf-8")
    
    saved_files["csv"] = csv_file
    logger.info(f"💾 CSV сохранён: {csv_file.name}")
    
    return saved_files


def save_comparison_stats(
    all_results: Dict[str, List[OCRResult]],
    output_dir: Path,
    experiment_name: str
) -> Path:
    """
    Сохранение сравнительной статистики всех моделей.
    
    Args:
        all_results: {model_name: [results]}
        output_dir: Папка для сохранения
        experiment_name: Название эксперимента
    
    Returns:
        Путь к файлу со статистикой
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    stats_list = []
    
    for model_name, results in all_results.items():
        if not results:
            continue
        
        stats = calculate_model_stats(results)
        if stats:
            stats_list.append({
                "model": model_name,
                "images": stats.total_images,
                "folders": stats.folders_processed,
                "total_time_sec": round(stats.total_time, 2),
                "avg_time_sec": round(stats.avg_time_per_image, 2),
                "throughput": round(stats.throughput, 3),
                "total_tokens": stats.total_tokens,
                "avg_tokens": round(stats.avg_tokens_per_image, 1),
                "total_chars": stats.total_chars,
                "avg_chars": round(stats.avg_chars_per_image, 1),
                "success_rate": round(stats.success_rate, 2),
                "errors": stats.errors
            })
    
    if not stats_list:
        logger.warning("Нет данных для сохранения статистики")
        return None
    
    df = pd.DataFrame(stats_list)
    
    # Сортировка по throughput
    df = df.sort_values("throughput", ascending=False)
    
    comparison_file = output_dir / f"{experiment_name}_comparison.csv"
    df.to_csv(comparison_file, index=False, encoding="utf-8")
    
    logger.info(f"💾 Сравнительная статистика: {comparison_file.name}")
    
    return comparison_file


print("✅ Функции сохранения определены")


✅ Функции сохранения определены


In [6]:
# Загрузка изображений для тестирования.
"""
Эта ячейка вызывает функции load_images_* и создаёт IMAGE_PATHS и FOLDER_GROUPS.
"""

# Проверка существования папки
if not TEST_ROOT.exists():
    print(f"❌ Папка не найдена: {TEST_ROOT}")
    print(f"   Текущий путь: {TEST_ROOT}")
    print(f"   Создайте папку или измените TEST_ROOT в Ячейке 2")
    IMAGE_PATHS = []
    FOLDER_GROUPS = {}
else:
    # Загрузка в зависимости от режима
    print(f"📂 Загрузка изображений из: {TEST_ROOT}")
    print(f"🔧 Режим: {LOAD_MODE}")
    
    if LOAD_MODE == "flat":
        IMAGE_PATHS, FOLDER_GROUPS = load_images_flat(
            TEST_ROOT,
            extensions=IMAGE_EXTENSIONS,
            max_images=MAX_TOTAL_IMAGES
        )
    
    elif LOAD_MODE == "nested":
        IMAGE_PATHS, FOLDER_GROUPS = load_images_nested(
            TEST_ROOT,
            extensions=IMAGE_EXTENSIONS,
            max_per_folder=MAX_IMAGES_PER_FOLDER,
            max_total=MAX_TOTAL_IMAGES
        )
    
    elif LOAD_MODE == "recursive":
        IMAGE_PATHS, FOLDER_GROUPS = load_images_recursive(
            TEST_ROOT,
            extensions=IMAGE_EXTENSIONS,
            max_total=MAX_TOTAL_IMAGES
        )
    
    else:
        print(f"❌ Неизвестный режим: {LOAD_MODE}")
        print(f"   Доступные: 'flat', 'nested', 'recursive'")
        IMAGE_PATHS = []
        FOLDER_GROUPS = {}
    
    # Вывод результатов
    if not IMAGE_PATHS:
        print(f"\n❌ Изображения не найдены!")
        print(f"   Проверьте:")
        print(f"   1. Путь: {TEST_ROOT}")
        print(f"   2. Режим: {LOAD_MODE}")
        print(f"   3. Расширения: {IMAGE_EXTENSIONS}")
    else:
        print(f"\n{'='*70}")
        print(f"✅ ИЗОБРАЖЕНИЯ ЗАГРУЖЕНЫ")
        print(f"{'='*70}")
        print(f"  Всего изображений: {len(IMAGE_PATHS)}")
        print(f"  Папок/групп: {len(FOLDER_GROUPS)}")
        
        print(f"\n📋 Структура данных:")
        for i, (folder, paths) in enumerate(FOLDER_GROUPS.items(), 1):
            print(f"\n  📂 {i}. {folder} ({len(paths)} изображений)")
            for j, p in enumerate(paths[:3], 1):
                print(f"     {j}. {p.name}")
            if len(paths) > 3:
                print(f"     ... и ещё {len(paths) - 3}")
            
            if i >= 5:
                remaining = len(FOLDER_GROUPS) - 5
                if remaining > 0:
                    print(f"\n  ... и ещё {remaining} папок")
                break
        
        print(f"\n{'='*70}")
        print(f"✅ Готово к тестированию моделей!")
        print(f"{'='*70}")


📂 Загрузка изображений из: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/notebooks/vlm-ingestion-pipeline/test_images
🔧 Режим: nested
2025-12-20 01:23:57,045 - INFO - 📁 Найдено 3 подпапок
2025-12-20 01:23:57,046 - INFO -   📂 o-predelnom-mnogomernom-raspredelenii: 4 изображений
2025-12-20 01:23:57,047 - INFO -   📂 standartizatsiya-i-kachestvo-zhizni: 3 изображений
2025-12-20 01:23:57,047 - INFO -   📂 Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изображений
2025-12-20 01:23:57,047 - INFO - ✅ Всего загружено 12 изображений из 3 папок

✅ ИЗОБРАЖЕНИЯ ЗАГРУЖЕНЫ
  Всего изображений: 12
  Папок/групп: 3

📋 Структура данных:

  📂 1. o-predelnom-mnogomernom-raspredelenii (4 изображений)
     1. page_001.png
     2. page_002.png
     3. page_003.png
     ... и ещё 1

  📂 2. standartizatsiya-i-kachestvo-zhizni (3 изображений)
     1. page_001.png
     2. page_002.png
     3. page_003.png

  📂 3. Книга_Python_К_вершинам_мастерства_Рамальо_Лучано (5 изображений)
     1. page_001.png
  

In [7]:
# Проверка размеров изображений


print("📐 РАЗМЕРЫ ИЗОБРАЖЕНИЙ")
print("="*60)

total_pixels = 0
sizes = []

for img_path in IMAGE_PATHS[:]:  # Первые 10
    img = Image.open(img_path)
    pixels = img.width * img.height
    total_pixels += pixels
    sizes.append((img_path.name, img.width, img.height, pixels))
    print(f"{img_path.name[:40]:<40} {img.width:>5}x{img.height:<5} {pixels/1e6:.1f}MP")

if sizes:
    avg_pixels = total_pixels / len(sizes)
    print(f"\n{'='*60}")
    print(f"Среднее: {avg_pixels/1e6:.1f} мегапикселей")
    
    if avg_pixels > 4e6:
        print(f"\n⚠️  ИЗОБРАЖЕНИЯ БОЛЬШИЕ (>{avg_pixels/1e6:.0f}MP)")
        print(f"   Рекомендуется resize до 1920px")
        print(f"   Это ускорит обработку в 2-3 раза!")


📐 РАЗМЕРЫ ИЗОБРАЖЕНИЙ
page_001.png                              1211x1713  2.1MP
page_002.png                              1211x1713  2.1MP
page_003.png                              1211x1713  2.1MP
page_004.png                              1211x1713  2.1MP
page_001.png                              1241x1755  2.2MP
page_002.png                              1241x1755  2.2MP
page_003.png                              1241x1755  2.2MP
page_001.png                               771x1114  0.9MP
page_002.png                               778x1105  0.9MP
page_003.png                               778x1105  0.9MP
page_004.png                               774x1082  0.8MP
page_005.png                               774x1082  0.8MP

Среднее: 1.6 мегапикселей


In [8]:
# Функции параллельной обработки
"""

АРХИТЕКТУРА ПАРАЛЛЕЛЬНОЙ ОБРАБОТКИ:

┌─────────────────────────────────────────────────────────────────────────┐
│                         ГЛАВНЫЙ ПОТОК                                   │
│  ┌──────────────────────────────────────────────────────────────────┐  │
│  │  test_model_parallel(model_name, max_workers=8)                  │  │
│  │  1. Подготовка списка задач (tasks)                              │  │
│  │  2. Создание ThreadPoolExecutor с max_workers потоками            │  │
│  │  3. Отправка задач в пул                                          │  │
│  └───────────────────┬──────────────────────────────────────────────┘  │
│                      │                                                   │
│                      ▼                                                   │
│  ┌─────────────────────────────────────────────────────────────────┐   │
│  │            ThreadPoolExecutor (пул потоков)                      │   │
│  │  ┌──────────┬──────────┬──────────┬──────────┬─────────┐         │   │
│  │  │ Worker 1 │ Worker 2 │ Worker 3 │ Worker 4 │   ...   │         │   │
│  │  └────┬─────┴────┬─────┴────┬─────┴────┬─────┴────┬────┘         │   │
│  │       │          │          │          │          │               │   │
│  │       ▼          ▼          ▼          ▼          ▼               │   │
│  │   API req 1  API req 2  API req 3  API req 4  API req 5          │   │
│  │       │          │          │          │          │               │   │
│  │       ▼          ▼          ▼          ▼          ▼               │   │
│  │   ┌──────────────────────────────────────────────────┐            │   │
│  │   │         vLLM Server (localhost:8000)             │            │   │
│  │   │  • Обрабатывает batch из нескольких запросов     │            │   │
│  │   │  • max_num_seqs=8-32 запросов одновременно       │            │   │
│  │   └──────────────────────────────────────────────────┘            │   │
│  │       │          │          │          │          │               │   │
│  │       ▼          ▼          ▼          ▼          ▼               │   │
│  │   Result 1   Result 2   Result 3   Result 4   Result 5           │   │
│  └───────┬──────────┬──────────┬──────────┬──────────┬──────────────┘   │
│          │          │          │          │          │                   │
│          └──────────┴──────────┴──────────┴──────────┘                   │
│                              │                                            │
│                              ▼                                            │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │  as_completed() - собирает результаты по мере готовности        │    │
│  │  • Не ждёт завершения всех задач                                │    │
│  │  • Обрабатывает результаты сразу после получения                │    │
│  │  • Обновляет прогресс-бар (tqdm)                                 │    │
│  └─────────────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────────────┘

КЛЮЧЕВЫЕ ОСОБЕННОСТИ:

1. **I/O-BOUND оптимизация**: Используем потоки (threads), т.к. задача 
   состоит в ожидании ответа от API. GIL освобождается при I/O операциях!

2. **Без блокировок (locks)**: Каждый worker обрабатывает свою задачу 
   независимо. Результаты собираются через Future objects (thread-safe).

3. **Динамическое планирование**: as_completed() возвращает результаты 
   по мере готовности, а не в порядке отправки.

4. **Контроль ресурсов**: max_workers ограничивает количество одновременных
   запросов, предотвращая перегрузку сервера.

ПАРАМЕТРЫ:

- max_workers=8: Рекомендуется для max_num_seqs=8-16 на сервере
- max_workers=16: Для max_num_seqs=32 и мощных GPU
- max_workers=4: Для стабильности при проблемах

ОГРАНИЧЕНИЯ:

- НЕ подходит для CPU-bound задач (используйте ProcessPoolExecutor)
- Память: каждый поток держит копию изображения в base64
- vLLM сервер должен поддерживать достаточный max_num_seqs
"""

# ============================================================================
# THREAD-SAFE результаты (опционально)
# ============================================================================
# Если нужна thread-safe коллекция результатов:
# results_lock = threading.Lock()
# Но в нашем случае Future.result() уже thread-safe!

# ============================================================================
# ФУНКЦИЯ 1: Проверка доступности модели
# ============================================================================
def check_model_availability(port: int, timeout: int = 5) -> bool:
    """
    Проверяет доступность vLLM сервера.
    
    Thread-safety: ✅ Безопасна (только чтение)
    
    Args:
        port: Порт сервера
        timeout: Таймаут проверки (секунды)
    
    Returns:
        True если сервер доступен, False иначе
    """
    try:
        response = requests.get(
            f"http://localhost:{port}/v1/models",
            timeout=timeout
        )
        return response.status_code == 200
    except Exception as e:
        logger.error(f"Порт {port} недоступен: {e}")
        return False


# ============================================================================
# ФУНКЦИЯ 2: Конвертация изображения в base64
# ============================================================================
def image_to_base64(image: Image.Image, max_size: int = 1920) -> str:
    """
    Конвертирует PIL Image в base64 с автоматическим resize.
    
    Thread-safety: ✅ Безопасна (работает с копией изображения)
    
    ОПТИМИЗАЦИЯ:
    - Resize до max_size для ускорения обработки
    - JPEG для RGB (меньше размер, быстрее передача)
    - PNG для RGBA (сохранение прозрачности)
    
    Args:
        image: PIL Image объект
        max_size: Максимальный размер по большей стороне
    
    Returns:
        Base64 строка
    """
    # Resize если изображение слишком большое
    if max(image.size) > max_size:
        ratio = max_size / max(image.size)
        new_size = (int(image.width * ratio), int(image.height * ratio))
        image = image.resize(new_size, Image.Resampling.LANCZOS)
        logger.debug(f"Изображение уменьшено: {image.size} → {new_size}")
    
    buffered = BytesIO()
    
    # Оптимизация формата
    if image.mode in ('RGBA', 'LA', 'P'):
        # PNG для изображений с прозрачностью
        image.save(buffered, format="PNG", optimize=True)
    else:
        # JPEG для RGB (быстрее и меньше)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image.save(buffered, format="JPEG", quality=95, optimize=True)
    
    return base64.b64encode(buffered.getvalue()).decode()


# ============================================================================
# ФУНКЦИЯ 3: OCR одного изображения (worker функция)
# ============================================================================
def ocr_single_image(
    img_path: Path,
    folder_name: str,
    model_name: str,
    model_config: Dict[str, Any],
    prompt_type: str,
    page_num: int
) -> OCRResult:
    """
    Обрабатывает одно изображение через OCR API.
    
    ⚡ WORKER ФУНКЦИЯ для ThreadPoolExecutor ⚡
    
    Thread-safety: ✅ Полностью безопасна
    - Каждый вызов работает со своими данными
    - Нет общих изменяемых переменных
    - API клиент создаётся локально
    
    ВАЖНО: Эта функция будет вызываться одновременно из разных потоков!
    
    Args:
        img_path: Путь к изображению
        folder_name: Название папки
        model_name: Название модели
        model_config: Конфигурация модели
        prompt_type: Тип промпта
        page_num: Номер страницы
    
    Returns:
        OCRResult с результатами обработки
    """
    start_time = time.time()
    
    try:
        # 1. Загрузка изображения
        image = Image.open(img_path)
        
        # 2. Конвертация в base64
        image_base64 = image_to_base64(image, max_size=1920)
        
        # 3. Создание API клиента (локального для потока)
        # Каждый поток создаёт свой клиент - это thread-safe!
        client = OpenAI(
            base_url=f"http://localhost:{model_config['port']}/v1",
            api_key="EMPTY",
            timeout=120  # Таймаут для долгих запросов
        )
        
        # 4. Формирование запроса
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": model_config["prompts"][prompt_type]
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_base64}"
                        }
                    }
                ]
            }
        ]
        
        # 5. Отправка запроса к vLLM
        # Блокирующий вызов, но GIL освобождается при I/O!
        response = client.chat.completions.create(
            model=model_config["model_path"],
            messages=messages,
            temperature=0.0,
            max_tokens=4096
        )
        
        # 6. Извлечение результата
        text = response.choices[0].message.content
        tokens = response.usage.total_tokens if response.usage else 0
        
        processing_time = time.time() - start_time
        
        # 7. Формирование результата
        return OCRResult(
            model_name=model_name,
            folder_name=folder_name,
            filename=img_path.name,
            page_number=page_num,
            text=text,
            tokens=tokens,
            processing_time=processing_time,
            text_length=len(text),
            timestamp=datetime.now().isoformat(),
            error=None
        )
        
    except Exception as e:
        # Обработка ошибок в worker потоке
        processing_time = time.time() - start_time
        logger.error(f"Ошибка обработки {img_path.name}: {e}")
        
        return OCRResult(
            model_name=model_name,
            folder_name=folder_name,
            filename=img_path.name,
            page_number=page_num,
            text="",
            tokens=0,
            processing_time=processing_time,
            text_length=0,
            timestamp=datetime.now().isoformat(),
            error=str(e)
        )


# ============================================================================
# ФУНКЦИЯ 4: Wrapper для ThreadPoolExecutor
# ============================================================================
def ocr_single_image_wrapper(args: Tuple) -> OCRResult:
    """
    Wrapper для распаковки аргументов (нужен для executor.map).
    
    Thread-safety: ✅ Безопасен (просто распаковывает tuple)
    
    Args:
        args: Tuple с аргументами для ocr_single_image
    
    Returns:
        OCRResult
    """
    return ocr_single_image(*args)


# ============================================================================
# ФУНКЦИЯ 5: Параллельное тестирование модели (ГЛАВНАЯ)
# ============================================================================
def test_model_parallel(
    model_name: str,
    max_workers: int = 8
) -> Optional[List[OCRResult]]:
    """
    ⚡⚡⚡ ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ МОДЕЛИ ⚡⚡⚡
    
    АЛГОРИТМ:
    
    1. ПОДГОТОВКА (главный поток):
       - Проверка доступности модели
       - Формирование списка задач (tasks)
       - Каждая задача = tuple с параметрами для worker
    
    2. ПАРАЛЛЕЛЬНАЯ ОБРАБОТКА (пул потоков):
       ┌─────────────────────────────────────────────┐
       │  with ThreadPoolExecutor(max_workers) as ex: │
       │                                              │
       │  1. Создаётся пул из max_workers потоков     │
       │  2. Задачи распределяются между workers      │
       │  3. Каждый worker вызывает ocr_single_image  │
       │  4. Результаты возвращаются через Future     │
       └─────────────────────────────────────────────┘
    
    3. СБОР РЕЗУЛЬТАТОВ (главный поток):
       - as_completed() итерируется по завершённым Future
       - Результаты добавляются в список сразу после получения
       - tqdm показывает прогресс в реальном времени
    
    4. ФИНАЛИЗАЦИЯ (главный поток):
       - Сохранение результатов
       - Подсчёт статистики
       - Вывод отчёта
    
    СИНХРОНИЗАЦИЯ:
    - Future objects автоматически thread-safe (concurrent.futures)
    - Каждый worker работает с локальными данными
    - Нет общих изменяемых переменных → НЕТ RACE CONDITIONS!
    
    ПАМЯТЬ:
    - Каждый поток: ~5-10 MB (зависит от размера изображения в base64)
    - max_workers=8: ~40-80 MB дополнительно
    - max_workers=16: ~80-160 MB дополнительно
    
    ПРОИЗВОДИТЕЛЬНОСТЬ:
    - I/O-bound: GIL освобождается при сетевых запросах
    - Ускорение: ~(max_workers / 1.5) раз (с учётом overhead)
    - Пример: max_workers=8 → ускорение ~5x
    
    Args:
        model_name: Название модели из MODELS_CONFIG
        max_workers: Количество параллельных потоков
                     Рекомендации:
                     - 4-8: Стабильная работа
                     - 8-16: Максимальная скорость (если vLLM справляется)
                     - 32+: Overkill, возможны проблемы с vLLM
    
    Returns:
        List[OCRResult] или None при ошибке
    """
    
    # ═══════════════════════════════════════════════════════════════════════
    # ЭТАП 1: ВАЛИДАЦИЯ И ПОДГОТОВКА
    # ═══════════════════════════════════════════════════════════════════════
    
    # Проверка загрузки изображений
    if 'IMAGE_PATHS' not in globals() or not IMAGE_PATHS:
        print(f"❌ Изображения не загружены! Запустите Ячейку 4")
        return None
    
    # Проверка конфигурации модели
    if model_name not in MODELS_CONFIG:
        print(f"❌ Модель {model_name} не найдена в MODELS_CONFIG")
        return None
    
    model_config = MODELS_CONFIG[model_name]
    
    # Проверка что модель включена
    if not model_config["enabled"]:
        print(f"⏸️  {model_name} отключена в конфигурации")
        return None
    
    # Проверка доступности сервера
    if not check_model_availability(model_config["port"]):
        print(f"❌ {model_name} недоступна на порту {model_config['port']}")
        print(f"   Запустите сервер: ./scripts/vllm/rocm/start_vllm_rocm.sh")
        return None
    
    # Заголовок
    print(f"{'='*70}")
    print(f"🚀 ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ: {model_name}")
    print(f"{'='*70}")
    print(f"✅ Модель доступна на порту {model_config['port']}")
    print(f"📝 Промпт: {model_config['prompts'][PROMPT_TYPE]}")
    print(f"📊 Изображений: {len(IMAGE_PATHS)}")
    print(f"⚡ Параллельных потоков: {max_workers}")
    print(f"{'='*70}\n")
    
    # ═══════════════════════════════════════════════════════════════════════
    # ЭТАП 2: ПОДГОТОВКА ЗАДАЧ
    # ═══════════════════════════════════════════════════════════════════════
    
    tasks = []
    for folder_name, folder_paths in FOLDER_GROUPS.items():
        for page_num, img_path in enumerate(folder_paths, 1):
            # Каждая задача = tuple с параметрами
            task = (
                img_path,
                folder_name,
                model_name,
                model_config,
                PROMPT_TYPE,
                page_num
            )
            tasks.append(task)
    
    print(f"📦 Подготовлено задач: {len(tasks)}")
    print(f"   Worker будет обрабатывать: {len(tasks) / max_workers:.1f} задач в среднем")
    print(f"\n{'─'*70}\n")
    
    # ═══════════════════════════════════════════════════════════════════════
    # ЭТАП 3: ПАРАЛЛЕЛЬНАЯ ОБРАБОТКА
    # ═══════════════════════════════════════════════════════════════════════
    
    start_time = time.time()
    results = []
    
    # Создание пула потоков
    # with автоматически закроет executor после завершения
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        
        # Отправка всех задач в пул
        # executor.submit возвращает Future object
        future_to_task = {
            executor.submit(ocr_single_image_wrapper, task): task
            for task in tasks
        }
        
        # Прогресс-бар
        with tqdm(total=len(tasks), desc=f"⚡ {model_name}", unit="img") as pbar:
            
            # as_completed() возвращает Future по мере готовности
            # НЕ в порядке отправки!
            for future in as_completed(future_to_task):
                task = future_to_task[future]
                
                try:
                    # Получение результата (блокирующий вызов)
                    result = future.result()
                    results.append(result)
                    
                    # Обновление прогресс-бара
                    status = "✅" if not result.error else "❌"
                    pbar.set_postfix({
                        'файл': result.filename[:20],
                        'время': f"{result.processing_time:.1f}с",
                        'символов': result.text_length,
                        'статус': status
                    })
                    
                except Exception as exc:
                    # Обработка неожиданных ошибок
                    logger.error(f"Worker exception: {exc}")
                    img_path = task[0]
                    results.append(OCRResult(
                        model_name=model_name,
                        folder_name=task[1],
                        filename=img_path.name,
                        page_number=task[5],
                        text="",
                        tokens=0,
                        processing_time=0,
                        text_length=0,
                        timestamp=datetime.now().isoformat(),
                        error=f"Worker exception: {exc}"
                    ))
                
                pbar.update(1)
    
    # Executor автоматически ждёт завершения всех задач перед выходом из with
    
    total_time = time.time() - start_time
    
    # ═══════════════════════════════════════════════════════════════════════
    # ЭТАП 4: СОХРАНЕНИЕ И СТАТИСТИКА
    # ═══════════════════════════════════════════════════════════════════════
    
    # Сохранение результатов
    ALL_RESULTS[model_name] = results
    
    print(f"\n💾 Сохранение результатов...")
    saved_files = save_results_to_files(results, OUTPUT_DIR, model_name)
    
    # Подсчёт статистики
    stats = calculate_model_stats(results)
    
    # Вывод отчёта
    print(f"\n{'='*70}")
    print(f"📊 СТАТИСТИКА {model_name}")
    print(f"{'='*70}")
    print(f"  Изображений:      {stats.total_images}")
    print(f"  Папок:            {stats.folders_processed}")
    print(f"  Общее время:      {stats.total_time:.1f}с ({stats.total_time/60:.1f} мин)")
    print(f"  Throughput:       {stats.throughput:.2f} изобр/с")
    print(f"  Среднее время:    {stats.avg_time_per_image:.2f}с/изобр")
    
    # Оценка ускорения
    theoretical_speedup = max_workers / 1.5  # С учётом overhead
    actual_speedup = (stats.avg_time_per_image * len(results)) / stats.total_time
    print(f"  Ускорение:        {actual_speedup:.1f}x (теор. {theoretical_speedup:.1f}x) ⚡")
    
    print(f"  Токенов:          {stats.total_tokens:,}")
    print(f"  Символов:         {stats.total_chars:,}")
    print(f"  Успешно:          {stats.success_rate:.1f}%")
    
    if stats.errors > 0:
        print(f"  ⚠️  Ошибок:       {stats.errors}")
    
    # Статистика по папкам
    if len(FOLDER_GROUPS) > 1:
        print(f"\n📂 По папкам:")
        for folder_name in FOLDER_GROUPS.keys():
            folder_stats = calculate_folder_stats(results, folder_name)
            if folder_stats:
                print(f"  {folder_name}: {folder_stats.total_images} изобр, "
                      f"{folder_stats.avg_time_per_image:.2f}с/изобр, "
                      f"{folder_stats.avg_chars_per_image:.0f} симв/изобр")
    
    # Сохранённые файлы
    print(f"\n💾 Файлы: {len(saved_files)}")
    for key, filepath in saved_files.items():
        print(f"  📄 {filepath.relative_to(PROJECT_ROOT)}")
    
    print(f"\n{'='*70}")
    print(f"✅ ЗАВЕРШЕНО!")
    print(f"{'='*70}\n")
    
    return results


# ============================================================================
# ФУНКЦИЯ 6: Алиас для обратной совместимости
# ============================================================================
def test_model(model_name: str, max_workers: int = 8):
    """
    Алиас для test_model_parallel с умными настройками.
    
    Автоматически выбирает max_workers на основе количества изображений:
    - <= 4 изображения: max_workers = 4
    - 5-12 изображений: max_workers = 8
    - > 12 изображений: max_workers = 16
    """
    if 'IMAGE_PATHS' in globals():
        num_images = len(IMAGE_PATHS)
        if num_images <= 4:
            max_workers = 4
        elif num_images <= 12:
            max_workers = 8
        else:
            max_workers = 16
    
    return test_model_parallel(model_name, max_workers=max_workers)


print("✅ Функции параллельной обработки определены")
print(f"   • ocr_single_image() - worker функция")
print(f"   • test_model_parallel() - главная функция с ThreadPoolExecutor")
print(f"   • test_model() - алиас с автоматическим выбором workers")


✅ Функции параллельной обработки определены
   • ocr_single_image() - worker функция
   • test_model_parallel() - главная функция с ThreadPoolExecutor
   • test_model() - алиас с автоматическим выбором workers


---

In [9]:
test_model_parallel("HunyuanOCR", max_workers=2)

🚀 ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ: HunyuanOCR
✅ Модель доступна на порту 8000
📝 Промпт: OCR this text. Extract all text accurately.
📊 Изображений: 12
⚡ Параллельных потоков: 2

📦 Подготовлено задач: 12
   Worker будет обрабатывать: 6.0 задач в среднем

──────────────────────────────────────────────────────────────────────



⚡ HunyuanOCR:   0%|          | 0/12 [00:00<?, ?img/s]

2025-12-19 00:27:20,096 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR:   8%|▊         | 1/12 [00:21<03:52, 21.14s/img, файл=page_001.png, время=21.1с, символов=3294, статус=✅]

2025-12-19 00:27:20,311 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


...

⚡ HunyuanOCR:  92%|█████████▏| 11/12 [01:46<00:06,  6.58s/img, файл=page_004.png, время=9.0с, символов=1996, статус=✅]

2025-12-19 00:28:49,249 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR: 100%|██████████| 12/12 [01:50<00:00,  9.19s/img, файл=page_005.png, время=10.7с, символов=2487, статус=✅]


💾 Сохранение результатов...


AttributeError: 'OCRResult' object has no attribute 'page_num'